### IS Proof of concept - DRMABS

In [1]:
import numpy as np
import pandas as pd
import itertools
import math
import string
import re
import pymongo

In [2]:
import time
from datetime import datetime
today = datetime.now()

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
# pip install selenium
from selenium import webdriver

In [4]:
from IPython.display import clear_output

In [5]:
# Files definition
root_path = 'Datasource'
urls_past_file = root_path+"/all_dairy_urls.csv"
comment_past_file = root_path+"/comment_DDCF.csv"

# จำนวนกระทู้ที่ต้องการให้ดึงใน 1 นาที
postNum = 7
postNumSleep = math.ceil(60/postNum)

In [6]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_comment = mydb["NIDA_PPSN_SCRAPE"]

### กรณียังไม่เคยดึง URL ที่มีการพูดถึง Keyword ใน Pantip

Get list of URL from search page

Scroll down to get all data (Crawler)

### กรณีเคยเก็บ URL แล้ว ให้เริ่มตรงนี้ได้เลย เพื่อไม่ให้ต้องทำ Request บ่อยๆ

Loop for scaping by URLs from file

In [7]:
dfURL = pd.read_csv(urls_past_file)
#dfURL = pd.concat([df_URLRead,df_URLRead1,df_URLRead2,df_URLRead3,df_URLRead4])
dfURL.drop_duplicates(keep='last',inplace=True,ignore_index=True)
dfURL.columns = ['URLs','Retrived_date']

In [8]:
dfURL.shape

(5152, 2)

In [9]:
def create_ListofDict_InsertMany(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
                "URLs": lx,
                "headline" : df.headline[c],
                "text": df.text[c],
                "article_date" : datetime.strptime(df.article_date[c], '%m/%d/%Y %H:%M:%S'),
                "Retrived_date": df.Retrived_date[c]
        }
        listofdict.append(info)
    return listofdict

In [12]:
startResult = 4947
endResult = 5152

# 0-1000 Complete
# 1001-2000 Complete
# 2001-3000 Complete
# 3001-4000 Complete
# 4001-5000
# 5001-5152

In [13]:
comments_list,time_list,error_list = [],[],[]

# รันรอบต่อไปให้ comment del many ออกด้วย
#col_comment.delete_many({})

for cnt,au in enumerate(dfURL.URLs[startResult:endResult],0):
    today = datetime.now()
    print('Processing URL no. ',startResult+cnt,' on ',today)
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(au)
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    try:
        div1 = soup.find("div", class_=lambda value: value and value=="display-post-status-leftside")
        headline_t = div1.find("h2", class_=lambda value: value and value=="display-post-title")
        for abbr in soup.find_all("abbr", class_=lambda value: value and value=="timeago"):
            time_list.append(abbr['data-utime'])
        for div3 in soup.find_all("div", class_=lambda value: value and value=="display-post-story"):
            if len(div3['class'])<2:
                comments_list.append([au,headline_t.text.strip(),str(div3.text).strip(),today])
        df_comments = pd.DataFrame(comments_list, columns=['URLs','headline', 'text', 'Retrived_date'])
        df_comments.insert(2,'article_date',value=time_list[0])
        process_data = create_ListofDict_InsertMany(df_comments)
        print(process_data)
        col_comment.insert_many(process_data)
        del df_comments
        comments_list.clear()
        time_list.clear()
    except Exception as err:
        print('กระทู้ไม่ถูกดึงเข้าระบบ', err)
        error_list.append([cnt,au,err])
    time.sleep(postNumSleep)
    driver.quit()
    clear_output(wait=True)

Processing URL no.  5151  on  2021-03-25 10:30:08.736844
[{'URLs': 'https://pantip.com/topic/30062751', 'headline': 'ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส  มีคำแนะนำครับ', 'text': 'หลายๆคนที่ไปออกกำลังกายที่ฟิตเนส\xa0\xa0ก็คงหวังว่าจะมีสุขภาพที่แข็งแรง\xa0\xa0มีรูปร่างที่ดีตามแบบของผู้ชายและผู้หญิง\nผมมีวิธีมาบอกครับรับรองได้ผลและปลอดภัย 100 %\xa0\xa0คือผมก็เป็นเทรนเนอร์แต่จรรยาบรรณของผมไม่เหมือนใคร\xa0\xa0\n\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 คนไทยส่วนใหญ่ไม่มีความเข้าใจถึงวิธีออกกำลังกายที่ถูกต้อง\xa0\xa0มัวไปหลงเชื่อแต่โฆษณาให้ทำโน่นทำนี่กินโน่นกินนี่\nผมจะบอกให้ว่ามันไม่มีหลอกนะครับยาวิเศษอาหารวิเศษอ่ะครับ\xa0\xa0การจะให้สุขภาพดีและร่างกายแข็งแรงเราต้องออกกำลังกายให้ถูกวิธี\nแล้วทานอาหารให้เป็นประโยชน์\n\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0ลดไขมันหรือลดน้ำหนัก\n\xa0\xa0\xa0\xa0\xa0\xa0 \n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0เมื่อเราพูดถึงการมีรูปร่างที่ดีขึ้น\xa0\xa0สิ่งแรกที่เรานึกคือการลดน้ำหนัก\xa0\xa0แต่แท้ที่จริงคือการทำให้ไขมันในร่างกายน้อยลง\xa0\xa0\nไม่ใช่น้ำหนัก